<a href="https://colab.research.google.com/github/SandyCOG/Aviation-Weather-Forecasting-Using-METAR-Data/blob/main/Aviation_Weather_Forecasting_Using_METAR_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install metpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 18.2 MB/s eta 0:00:00


In [5]:
import metpy.io

In [11]:
# Import necessary modules
from metpy.io.metar import Metar

from metpy.io.metar import parse_metar


In [7]:
metpy.io.parse_metar_file("/content/Aviation Weather Forecasting Using METAR Data Set.txt", year=2014, month='January')

KeyError: "None of ['station_id'] are in the columns"

In [12]:
# Example METAR string
metar_string = "KMIA 181353Z 15013KT 10SM SCT025 BKN250 27/21 A3006 RMK AO2 SLP178 T02670206"

# Parse METAR using MetPy MetarParser
parser = Metar()
metar = parser.parse_metar(metar_string)

# Print parsed information
print(f"Station: {metar.station}")
print(f"Time: {metar.time}")
print(f"Temperature: {metar.temperature}")
print(f"Dew Point: {metar.dewpoint}")
print(f"Wind: {metar.wind_direction} degrees at {metar.wind_speed} knots")
print(f"Visibility: {metar.visibility} statute miles")


TypeError: metar.__new__() missing 28 required positional arguments: 'station_id', 'latitude', 'longitude', 'elevation', 'date_time', 'wind_direction', 'wind_speed', 'wind_gust', 'visibility', 'current_wx1', 'current_wx2', 'current_wx3', 'skyc1', 'skylev1', 'skyc2', 'skylev2', 'skyc3', 'skylev3', 'skyc4', 'skylev4', 'cloudcover', 'temperature', 'dewpoint', 'altimeter', 'current_wx1_symbol', 'current_wx2_symbol', 'current_wx3_symbol', and 'remarks'

In [ ]:
import pandas as pd


In [15]:
import pandas as pd
df = pd.read_csv("/content/Aviation Weather Forecasting Using METAR Data Set.txt", sep=",")
df


,airport_id,date,metar
0,KMIA,2014-01-01,KMIA 010053Z 04009G15KT 10SM OVC025 23/18 A302...
1,KMIA,2014-01-01,KMIA 010153Z 04009G15KT 10SM OVC023 23/18 A302...
2,KMIA,2014-01-01,KMIA 010253Z 05009G19KT 10SM -RA OVC025 23/18 ...
3,KMIA,2014-01-01,KMIA 010353Z 07010KT 10SM OVC028 23/17 A3021 R...
4,KMIA,2014-01-01,KMIA 010453Z 07010KT 10SM OVC028 24/18 A3019 R...
...,...,...,...
89585,KMIA,2023-12-31,KMIA 311953Z 28004KT 10SM FEW250 21/08 A3019 R...
89586,KMIA,2023-12-31,KMIA 312053Z 31005KT 10SM FEW250 21/09 A3019 R...
89587,KMIA,2023-12-31,KMIA 312153Z 29004KT 10SM FEW250 21/08 A3019 R...
89588,KMIA,2023-12-31,KMIA 312253Z 29004KT 10SM CLR 19/09 A3021 RMK ...


In [17]:
def parse_metar(metar_string):
    parser = Metar()
    metar = parser.parse_metar(metar_string)
    return metar

# Apply the parse_metar function to the "metar" column
df['parsed_metar'] = df['metar'].apply(parse_metar)

# Extract relevant information into new columns
df['wind_direction'] = df['parsed_metar'].apply(lambda x: x.wind_direction if x else None)
df['wind_speed'] = df['parsed_metar'].apply(lambda x: x.wind_speed if x else None)
df['visibility'] = df['parsed_metar'].apply(lambda x: x.visibility if x else None)
df['temperature'] = df['parsed_metar'].apply(lambda x: x.temperature if x else None)
df['dew_point'] = df['parsed_metar'].apply(lambda x: x.dewpoint if x else None)

# Drop the 'parsed_metar' column if no longer needed
df = df.drop('parsed_metar', axis=1)

# Display the resulting DataFrame
print(df)

TypeError: metar.__new__() missing 28 required positional arguments: 'station_id', 'latitude', 'longitude', 'elevation', 'date_time', 'wind_direction', 'wind_speed', 'wind_gust', 'visibility', 'current_wx1', 'current_wx2', 'current_wx3', 'skyc1', 'skylev1', 'skyc2', 'skylev2', 'skyc3', 'skylev3', 'skyc4', 'skylev4', 'cloudcover', 'temperature', 'dewpoint', 'altimeter', 'current_wx1_symbol', 'current_wx2_symbol', 'current_wx3_symbol', and 'remarks'

In [18]:
def extract_metar_info(metar_string):
    parsed_metar = parse_metar(metar_string)
    if parsed_metar:
        return {
            'wind_direction': parsed_metar.wind_direction.value,
            'wind_speed': parsed_metar.wind_speed.value,
            'visibility': parsed_metar.visibility.value,
            'temperature': parsed_metar.temperature.value,
            'dewpoint': parsed_metar.dewpoint.value
        }
    else:
        return None

# Apply the extract_metar_info function to the "metar" column
df['parsed_metar'] = df['metar'].apply(extract_metar_info)

# Extract relevant information into new columns
df['wind_direction'] = df['parsed_metar'].apply(lambda x: x['wind_direction'] if x else None)
df['wind_speed'] = df['parsed_metar'].apply(lambda x: x['wind_speed'] if x else None)
df['visibility'] = df['parsed_metar'].apply(lambda x: x['visibility'] if x else None)
df['temperature'] = df['parsed_metar'].apply(lambda x: x['temperature'] if x else None)
df['dewpoint'] = df['parsed_metar'].apply(lambda x: x['dewpoint'] if x else None)

# Drop the 'parsed_metar' column if no longer needed
df = df.drop('parsed_metar', axis=1)

# Display the resulting DataFrame
print(df)

TypeError: metar.__new__() missing 28 required positional arguments: 'station_id', 'latitude', 'longitude', 'elevation', 'date_time', 'wind_direction', 'wind_speed', 'wind_gust', 'visibility', 'current_wx1', 'current_wx2', 'current_wx3', 'skyc1', 'skylev1', 'skyc2', 'skylev2', 'skyc3', 'skylev3', 'skyc4', 'skylev4', 'cloudcover', 'temperature', 'dewpoint', 'altimeter', 'current_wx1_symbol', 'current_wx2_symbol', 'current_wx3_symbol', and 'remarks'

In [19]:
def extract_metar_info(metar_string):
    parsed_metar = parse_metar(metar_string)
    if parsed_metar:
        return {
            'wind_direction': parsed_metar.wind_direction,
            'wind_speed': parsed_metar.wind_speed,
            'visibility': parsed_metar.visibility,
            'temperature': parsed_metar.temperature,
            'dewpoint': parsed_metar.dewpoint
        }
    else:
        return None

# Apply the extract_metar_info function to the "metar" column
df['parsed_metar'] = df['metar'].apply(extract_metar_info)

# Extract relevant information into new columns
df['wind_direction'] = df['parsed_metar'].apply(lambda x: x['wind_direction'] if x else None)
df['wind_speed'] = df['parsed_metar'].apply(lambda x: x['wind_speed'] if x else None)
df['visibility'] = df['parsed_metar'].apply(lambda x: x['visibility'] if x else None)
df['temperature'] = df['parsed_metar'].apply(lambda x: x['temperature'] if x else None)
df['dewpoint'] = df['parsed_metar'].apply(lambda x: x['dewpoint'] if x else None)

# Drop the 'parsed_metar' column if no longer needed
df = df.drop('parsed_metar', axis=1)

# Display the resulting DataFrame
print(df)


TypeError: metar.__new__() missing 28 required positional arguments: 'station_id', 'latitude', 'longitude', 'elevation', 'date_time', 'wind_direction', 'wind_speed', 'wind_gust', 'visibility', 'current_wx1', 'current_wx2', 'current_wx3', 'skyc1', 'skylev1', 'skyc2', 'skylev2', 'skyc3', 'skylev3', 'skyc4', 'skylev4', 'cloudcover', 'temperature', 'dewpoint', 'altimeter', 'current_wx1_symbol', 'current_wx2_symbol', 'current_wx3_symbol', and 'remarks'